## 2.1 Import Python Libraries

In [ ]:
import boto3, cv2, time, numpy as np, matplotlib.pyplot as plt, random
from sagemaker.pytorch import PyTorchPredictor
from sagemaker.deserializers import JSONDeserializer

## 2.2 Check if Endpoint creation is successful and create the predictor

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

# Restore the endpoint name stored in the 2_DeployEndpoint.ipynb notebook
%store -r ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

endpoint_created = False
while True:
    response = sm_client.list_endpoints()
    for ep in response['Endpoints']:
        print(f"Endpoint Status = {ep['EndpointStatus']}")
        if ep['EndpointName']==ENDPOINT_NAME and ep['EndpointStatus']=='InService':
            endpoint_created = True
            break
    if endpoint_created:
        break
    time.sleep(5)

In [ ]:
predictor = PyTorchPredictor(endpoint_name=ENDPOINT_NAME,
                             deserializer=JSONDeserializer())

## 2.3 Run Inference and Generate output results

In [ ]:
import psutil
import numpy as np
import cv2
import boto3
import time
import random
import threading
from tqdm import tqdm
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Function to record CPU and memory usage over time
def record_usage(usage_stats):
    while keep_recording:
        cpu_percent = psutil.cpu_percent()
        memory_info = psutil.virtual_memory()
        usage_stats.append((time.time(), cpu_percent, memory_info.percent))
        time.sleep(1)  # Adjust sleep time as needed for finer or coarser measurements

# Function to start the monitoring thread
def start_monitoring():
    global keep_recording
    keep_recording = True
    usage_stats = []
    monitor_thread = threading.Thread(target=record_usage, args=(usage_stats,))
    monitor_thread.start()
    return usage_stats, monitor_thread

# Function to stop the monitoring thread
def stop_monitoring(monitor_thread):
    global keep_recording
    keep_recording = False
    monitor_thread.join()

In [ ]:
# # Setup S3 client and variables
# s3_client = boto3.client('s3')
# bucket_name = 'saabs'
# prefix = 'data/'
# response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
# global_image_paths = [x['Key'] for x in response['Contents']][1:]  # Skip first item if needed
# num_iters = 5


# # Main profiling loop
# for num_samples in [1, 25, 50]:
#     times = []
#     all_cpu_usages = []
#     all_memory_usages = []

#     for _ in tqdm(range(num_iters)):
#         # Start monitoring
#         usage_stats, monitor_thread = start_monitoring()

#         image_paths = random.sample(global_image_paths, num_samples)
#         payload_list = []
#         # Your image processing logic here
#         for file_name in image_paths:
#             # Download the image from S3
#             file_obj = s3_client.get_object(Bucket=bucket_name, Key=file_name)
#             file_content = file_obj['Body'].read()

#             # Convert bytes to numpy array
#             nparr = np.frombuffer(file_content, np.uint8)
#             orig_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

#             image_height, image_width, _ = orig_image.shape
#             model_height, model_width = 300, 300
#             x_ratio = image_width/model_width
#             y_ratio = image_height/model_height

#             resized_image = cv2.resize(orig_image, (model_height, model_width))
#             payload = cv2.imencode('.jpg', resized_image)[1].tobytes()
#             payload_list.append(payload)
            
#         total_infer_time_start = time.time()
        
#         for payload in payload_list:
#             result = predictor.predict(payload)

#             if 'boxes' in result:
#                 for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result['boxes']):
#                     # Draw Bounding Boxes
#                     x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
#                     y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
#                     color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                     cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
#                     cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#                     cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#                     if 'masks' in result:
#                         # Draw Masks
#                         mask = cv2.resize(np.asarray(result['masks'][idx]), dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
#                         for c in range(3):
#                             orig_image[:,:,c] = np.where(mask>0.5, orig_image[:,:,c]*(0.5)+0.5*color[c], orig_image[:,:,c])

#             if 'probs' in result:
#                 # Find Class
#                 lbl = result['probs'].index(max(result['probs']))
#                 color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                 cv2.putText(orig_image, f"Class: {int(lbl)}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

#             if 'keypoints' in result:
#                 # Define the colors for the keypoints and lines
#                 keypoint_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                 line_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))

#                 # Define the keypoints and the lines to draw
#                 # keypoints = keypoints_array[:, :, :2]  # Ignore the visibility values
#                 lines = [
#                     (0, 1), (0, 2), (1, 3), (2, 4),  # Head
#                     (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Torso
#                     (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
#                 ]

#                 # Draw the keypoints and the lines on the image
#                 for keypoints_instance in result['keypoints']:
#                     # Draw the keypoints
#                     for keypoint in keypoints_instance:
#                         if keypoint[2] == 0:  # If the keypoint is not visible, skip it
#                             continue
#                         cv2.circle(orig_image, (int(x_ratio*keypoint[:2][0]),int(y_ratio*keypoint[:2][1])), radius=5, color=keypoint_color, thickness=-1)

#                     # Draw the lines
#                     for line in lines:
#                         start_keypoint = keypoints_instance[line[0]]
#                         end_keypoint = keypoints_instance[line[1]]
#                         if start_keypoint[2] == 0 or end_keypoint[2] == 0:  # If any of the keypoints is not visible, skip the line
#                             continue
#                         cv2.line(orig_image, (int(x_ratio*start_keypoint[:2][0]),int(y_ratio*start_keypoint[:2][1])),(int(x_ratio*end_keypoint[:2][0]),int(y_ratio*end_keypoint[:2][1])), color=line_color, thickness=2)


#         total_infer_time_end = time.time()
#         times.append(total_infer_time_end - total_infer_time_start)

#         # Stop monitoring
#         stop_monitoring(monitor_thread)

#         # Extract CPU and memory usage over time
#         timestamps = [record[0] for record in usage_stats]
#         cpu_usages = [record[1] for record in usage_stats]
#         memory_usages = [record[2] for record in usage_stats]
#         all_cpu_usages.append(cpu_usages)
#         all_memory_usages.append(memory_usages)

#     # Output the profiling results for this batch size
#     print(f'Num samples: {num_samples}, mean time = {np.mean(times)}, std dev = {np.std(times)}')

In [ ]:
def process_image(payload):
    result = predictor.predict(payload)
    return result

In [ ]:
import os 
os.cpu_count()

In [ ]:
# Setup S3 client and variables
s3_client = boto3.client('s3')
bucket_name = 'saabs'
prefix = 'data/'
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
global_image_paths = [x['Key'] for x in response['Contents']][1:]  # Skip first item if needed
num_iters = 5


# Main profiling loop
for num_samples in [1, 5, 25, 50]:
    times = []
    all_cpu_usages = []
    all_memory_usages = []

    for _ in tqdm(range(num_iters)):
        # Start monitoring
        usage_stats, monitor_thread = start_monitoring()

        image_paths = random.sample(global_image_paths, num_samples)
        payload_list = []
        # Your image processing logic here
        for file_name in image_paths:
            # Download the image from S3
            file_obj = s3_client.get_object(Bucket=bucket_name, Key=file_name)
            file_content = file_obj['Body'].read()

            # Convert bytes to numpy array
            nparr = np.frombuffer(file_content, np.uint8)
            orig_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            image_height, image_width, _ = orig_image.shape
            model_height, model_width = 300, 300
            x_ratio = image_width/model_width
            y_ratio = image_height/model_height

            resized_image = cv2.resize(orig_image, (model_height, model_width))
            payload = cv2.imencode('.jpg', resized_image)[1].tobytes()
            payload_list.append(payload)
            
        total_infer_time_start = time.time()
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            results = executor.map(process_image, payload_list)
            
        total_infer_time_end = time.time()
        times.append(total_infer_time_end - total_infer_time_start)
        
        for result in results:
            if 'boxes' in result:
                for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result['boxes']):
                    # Draw Bounding Boxes
                    x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
                    y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
                    color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
                    cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
                    cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                    cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                    if 'masks' in result:
                        # Draw Masks
                        mask = cv2.resize(np.asarray(result['masks'][idx]), dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
                        for c in range(3):
                            orig_image[:,:,c] = np.where(mask>0.5, orig_image[:,:,c]*(0.5)+0.5*color[c], orig_image[:,:,c])

            if 'probs' in result:
                # Find Class
                lbl = result['probs'].index(max(result['probs']))
                color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
                cv2.putText(orig_image, f"Class: {int(lbl)}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

            if 'keypoints' in result:
                # Define the colors for the keypoints and lines
                keypoint_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
                line_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))

                # Define the keypoints and the lines to draw
                # keypoints = keypoints_array[:, :, :2]  # Ignore the visibility values
                lines = [
                    (0, 1), (0, 2), (1, 3), (2, 4),  # Head
                    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Torso
                    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
                ]

                # Draw the keypoints and the lines on the image
                for keypoints_instance in result['keypoints']:
                    # Draw the keypoints
                    for keypoint in keypoints_instance:
                        if keypoint[2] == 0:  # If the keypoint is not visible, skip it
                            continue
                        cv2.circle(orig_image, (int(x_ratio*keypoint[:2][0]),int(y_ratio*keypoint[:2][1])), radius=5, color=keypoint_color, thickness=-1)

                    # Draw the lines
                    for line in lines:
                        start_keypoint = keypoints_instance[line[0]]
                        end_keypoint = keypoints_instance[line[1]]
                        if start_keypoint[2] == 0 or end_keypoint[2] == 0:  # If any of the keypoints is not visible, skip the line
                            continue
                        cv2.line(orig_image, (int(x_ratio*start_keypoint[:2][0]),int(y_ratio*start_keypoint[:2][1])),(int(x_ratio*end_keypoint[:2][0]),int(y_ratio*end_keypoint[:2][1])), color=line_color, thickness=2)


        # total_infer_time_end = time.time()
        # times.append(total_infer_time_end - total_infer_time_start)

        # Stop monitoring
        stop_monitoring(monitor_thread)

        # Extract CPU and memory usage over time
        timestamps = [record[0] for record in usage_stats]
        cpu_usages = [record[1] for record in usage_stats]
        memory_usages = [record[2] for record in usage_stats]
        all_cpu_usages.append(cpu_usages)
        all_memory_usages.append(memory_usages)

    # Output the profiling results for this batch size
    print(f'Num samples: {num_samples}, mean time = {np.mean(times)}, std dev = {np.std(times)}')

In [ ]:
import matplotlib.pyplot as plt

num_samples_in_exp = [1, 5, 25, 50]

# Function to plot CPU and Memory usage
def plot_usages(cpu_usages, memory_usages, num_samples):
    # Assuming each entry in cpu_usages and memory_usages corresponds to a different trial
    fig, axs = plt.subplots(2, len(num_samples_in_exp), figsize=(15, 10))
    for i in range(len(num_samples_in_exp)):
        # CPU Usage plots
        axs[0, i].plot(cpu_usages[i])
        axs[0, i].set_title(f'CPU Usage for {num_samples[i]} images')
        axs[0, i].set_xlabel('Time (s)')
        axs[0, i].set_ylabel('CPU Usage (%)')
        
        # Memory Usage plots
        axs[1, i].plot(memory_usages[i])
        axs[1, i].set_title(f'Memory Usage for {num_samples[i]} images')
        axs[1, i].set_xlabel('Time (s)')
        axs[1, i].set_ylabel('Memory Usage (%)')

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Call the plotting function with your data
plot_usages(all_cpu_usages, all_memory_usages, num_samples_in_exp)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # Ensure numpy is imported

num_samples_in_exp = [1, 5, 25, 50]

# Updated function to plot CPU and Memory usage per iteration
def plot_usages(all_cpu_usages, all_memory_usages, num_samples, num_iters):
    num_cases = len(num_samples)  # The number of different image count cases
    iter_length = len(all_cpu_usages[0]) // num_iters  # Assuming all have same length and evenly divided
    
    # Setting up the subplot grid
    fig, axs = plt.subplots(num_iters * 2, num_cases, figsize=(15, 10), constrained_layout=True)

    # Looping through each case (1, 25, 50 images)
    for i in range(num_cases):
        # Looping through each iteration
        for j in range(num_iters):
            # Extracting the data for the current iteration
            cpu_data = all_cpu_usages[i][j * iter_length: (j + 1) * iter_length]
            mem_data = all_memory_usages[i][j * iter_length: (j + 1) * iter_length]

            # Plotting CPU usage
            cpu_ax = axs[j * 2, i]
            cpu_ax.plot(cpu_data)
            if j == 0:  # Only set title for the first row
                cpu_ax.set_title(f'CPU Usage for {num_samples[i]} images')
            cpu_ax.set_ylabel(f'CPU % (Iter {j+1})')
            cpu_ax.set_ylim([0, 100])  # Assuming CPU percentage
            if j == num_iters - 1:  # Only set x-label for the last row
                cpu_ax.set_xlabel('Time (s)')

            # Plotting Memory usage
            mem_ax = axs[j * 2 + 1, i]
            mem_ax.plot(mem_data)
            if j == num_iters - 1:  # Only set x-label for the last row
                mem_ax.set_xlabel('Time (s)')
            mem_ax.set_ylabel(f'Memory % (Iter {j+1})')
            mem_ax.set_ylim([0, 100])  # Assuming Memory percentage

    plt.show()

# Call the updated plotting function with your data
plot_usages(all_cpu_usages, all_memory_usages, num_samples_in_exp, num_iters)


In [ ]:
# #reading from s3
# s3_client = boto3.client('s3')
# bucket_name = 'saabs'
# prefix = 'data/'
# response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
# count = 0
# global_image_paths = [x['Key'] for x in response['Contents']]
# times = []
# global_image_paths = global_image_paths[1:]

# num_iters = 5

# for num_samples in [1, 25, 50]:
#     for _ in range(num_iters):
#         image_paths = random.sample(global_image_paths, num_samples)

#         total_infer_time_start = time.time()
#         for file_name in image_paths:
#             # Download the image from S3
#             file_obj = s3_client.get_object(Bucket=bucket_name, Key=file_name)
#             file_content = file_obj['Body'].read()

#             # Convert bytes to numpy array
#             nparr = np.frombuffer(file_content, np.uint8)
#             orig_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

#             image_height, image_width, _ = orig_image.shape
#             model_height, model_width = 300, 300
#             x_ratio = image_width/model_width
#             y_ratio = image_height/model_height

#             resized_image = cv2.resize(orig_image, (model_height, model_width))
#             payload = cv2.imencode('.jpg', resized_image)[1].tobytes()
#             result = predictor.predict(payload)

#             if 'boxes' in result:
#                 for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result['boxes']):
#                     # Draw Bounding Boxes
#                     x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
#                     y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
#                     color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                     cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
#                     cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#                     cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#                     if 'masks' in result:
#                         # Draw Masks
#                         mask = cv2.resize(np.asarray(result['masks'][idx]), dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
#                         for c in range(3):
#                             orig_image[:,:,c] = np.where(mask>0.5, orig_image[:,:,c]*(0.5)+0.5*color[c], orig_image[:,:,c])

#             if 'probs' in result:
#                 # Find Class
#                 lbl = result['probs'].index(max(result['probs']))
#                 color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                 cv2.putText(orig_image, f"Class: {int(lbl)}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

#             if 'keypoints' in result:
#                 # Define the colors for the keypoints and lines
#                 keypoint_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#                 line_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))

#                 # Define the keypoints and the lines to draw
#                 # keypoints = keypoints_array[:, :, :2]  # Ignore the visibility values
#                 lines = [
#                     (0, 1), (0, 2), (1, 3), (2, 4),  # Head
#                     (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Torso
#                     (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
#                 ]

#                 # Draw the keypoints and the lines on the image
#                 for keypoints_instance in result['keypoints']:
#                     # Draw the keypoints
#                     for keypoint in keypoints_instance:
#                         if keypoint[2] == 0:  # If the keypoint is not visible, skip it
#                             continue
#                         cv2.circle(orig_image, (int(x_ratio*keypoint[:2][0]),int(y_ratio*keypoint[:2][1])), radius=5, color=keypoint_color, thickness=-1)

#                     # Draw the lines
#                     for line in lines:
#                         start_keypoint = keypoints_instance[line[0]]
#                         end_keypoint = keypoints_instance[line[1]]
#                         if start_keypoint[2] == 0 or end_keypoint[2] == 0:  # If any of the keypoints is not visible, skip the line
#                             continue
#                         cv2.line(orig_image, (int(x_ratio*start_keypoint[:2][0]),int(y_ratio*start_keypoint[:2][1])),(int(x_ratio*end_keypoint[:2][0]),int(y_ratio*end_keypoint[:2][1])), color=line_color, thickness=2)

#         total_infer_time_end = time.time()
#         times.append(total_infer_time_end - total_infer_time_start)
#     print(f'Num samples: {num_samples}, mean = {np.mean(times)}, std dev = {np.std(times)}')

In [ ]:
# #reading from s3
# s3_client = boto3.client('s3')
# bucket_name = 'saabs'
# prefix = 'data/'
# response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
# count = 0
# image_paths = [x['Key'] for x in response['Contents']]
# times = []
# image_paths = image_paths[1:]

# total_infer_time_start = time.time()
# for file_name in image_paths:

#     infer_start_time = time.time()

#     # Download the image from S3
#     file_obj = s3_client.get_object(Bucket=bucket_name, Key=file_name)
#     file_content = file_obj['Body'].read()

#     # Convert bytes to numpy array
#     nparr = np.frombuffer(file_content, np.uint8)
#     orig_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

#     image_height, image_width, _ = orig_image.shape
#     model_height, model_width = 300, 300
#     x_ratio = image_width/model_width
#     y_ratio = image_height/model_height

#     resized_image = cv2.resize(orig_image, (model_height, model_width))
#     payload = cv2.imencode('.jpg', resized_image)[1].tobytes()
#     result = predictor.predict(payload)
#     # count += 1

#     if 'boxes' in result:
#         for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result['boxes']):
#             # Draw Bounding Boxes
#             x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
#             y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
#             color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#             cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
#             cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#             cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
#             if 'masks' in result:
#                 # Draw Masks
#                 mask = cv2.resize(np.asarray(result['masks'][idx]), dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
#                 for c in range(3):
#                     orig_image[:,:,c] = np.where(mask>0.5, orig_image[:,:,c]*(0.5)+0.5*color[c], orig_image[:,:,c])

#     if 'probs' in result:
#         # Find Class
#         lbl = result['probs'].index(max(result['probs']))
#         color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#         cv2.putText(orig_image, f"Class: {int(lbl)}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

#     if 'keypoints' in result:
#         # Define the colors for the keypoints and lines
#         keypoint_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
#         line_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))

#         # Define the keypoints and the lines to draw
#         # keypoints = keypoints_array[:, :, :2]  # Ignore the visibility values
#         lines = [
#             (0, 1), (0, 2), (1, 3), (2, 4),  # Head
#             (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Torso
#             (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
#         ]

#         # Draw the keypoints and the lines on the image
#         for keypoints_instance in result['keypoints']:
#             # Draw the keypoints
#             for keypoint in keypoints_instance:
#                 if keypoint[2] == 0:  # If the keypoint is not visible, skip it
#                     continue
#                 cv2.circle(orig_image, (int(x_ratio*keypoint[:2][0]),int(y_ratio*keypoint[:2][1])), radius=5, color=keypoint_color, thickness=-1)

#             # Draw the lines
#             for line in lines:
#                 start_keypoint = keypoints_instance[line[0]]
#                 end_keypoint = keypoints_instance[line[1]]
#                 if start_keypoint[2] == 0 or end_keypoint[2] == 0:  # If any of the keypoints is not visible, skip the line
#                     continue
#                 cv2.line(orig_image, (int(x_ratio*start_keypoint[:2][0]),int(y_ratio*start_keypoint[:2][1])),(int(x_ratio*end_keypoint[:2][0]),int(y_ratio*end_keypoint[:2][1])), color=line_color, thickness=2)

#     # plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
#     # plt.show()

#     infer_end_time = time.time()
#     times.append(infer_end_time - infer_start_time)

#     # print(f"Inference Time = {infer_end_time - infer_start_time:0.4f} seconds")
# # times = np.array(times)
# total_infer_time_end = time.time()
# print(total_infer_time_end - total_infer_time_start)
# print(np.mean(times))
# print(np.std(times))

## 2.4 Cleanup by removing Endpoint, Endpoint Config and Model

In [ ]:
response = sm_client.describe_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
print(response)
endpoint_config_name = response['EndpointConfigName']

# Delete Endpoint
sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)

# Delete Endpoint Configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete Model
for prod_var in response['ProductionVariants']:
    model_name = prod_var['ModelName']
    sm_client.delete_model(ModelName=model_name)     